Importing necessary libraries

In [ ]:
import pandas as pd
import math

Reading csv files

In [ ]:
# Load the synthetic datasets
df_records = pd.read_csv('synthetic_records_huge.csv')
df_params = pd.read_csv('synthetic_parameters_huge.csv')

In [ ]:
df_records.head(5)

,time,sender,receiver,success
0,41,D,A,1
1,16,H,E,1
2,44,X,R,1
3,28,B,A,1
4,15,Q,U,1


In [ ]:
df_params.head(5)

,T_window,n_periods,decay_rate,u,confidence_threshold,alpha,default_trust
0,47,8,0.72,0.48,9,0.34,0.72
1,49,7,0.80,0.32,5,0.40,0.85
2,29,10,0.71,0.43,7,0.55,0.82
3,27,3,0.90,0.16,7,0.78,0.85
4,35,4,0.86,0.46,9,0.39,0.87


In [ ]:
# select the first row of parameters
params = df_params.iloc[0]
T_window = params['T_window']
n_periods = params['n_periods']
decay_rate = params['decay_rate']
u = params['u']
confidence_threshold = params['confidence_threshold']
alpha = params['alpha']
default_trust = params['default_trust']

# Set the current simulation time as the maximum time in the records
current_time = df_records['time'].max()

# Convert records DataFrame to list of dictionaries for processing
records = df_records.to_dict('records')
records = sorted(records,key=lambda x:x['time'])
records[0:5]

[{'time': 1, 'sender': 'Y', 'receiver': 'F', 'success': 1},
 {'time': 1, 'sender': 'D', 'receiver': 'C', 'success': 0},
 {'time': 1, 'sender': 'Q', 'receiver': 'S', 'success': 1},
 {'time': 1, 'sender': 'J', 'receiver': 'K', 'success': 1},
 {'time': 1, 'sender': 'S', 'receiver': 'J', 'success': 1}]

Divide the sliding window [current_time - T_window, current_time] into n_periods equal sub-intervals.


In [ ]:
def get_period_boundaries(current_time, T_window, n_periods):
    start_time = current_time - T_window
    period_length = T_window / n_periods
    boundaries = []
    for i in range(int(n_periods)):
        period_start = start_time + i * period_length
        period_end = period_start + period_length
        boundaries.append((period_start, period_end))
    return boundaries

Compute the decay weight based on the difference between the period midpoint and current time.

In [ ]:
def period_decay(midpoint, current_time, decay_rate):
    time_diff = current_time - midpoint
    return decay_rate ** time_diff

  Filter records whose time falls within the period [period_start, period_end].

In [ ]:
def get_records_in_period(records, period_start, period_end):
    return [rec for rec in records if period_start <= rec["time"] <= period_end]



*   Compute direct trust in a period between node_i and node_j.
*   Bayesian trust = (Ns+1)/(Ns+Nf+2) multiplied by a penalty factor (1-u)^Nf.
*   Returns: (direct_trust, total_interactions) for the period.


In [ ]:
def compute_direct_trust_in_period(records, node_i, node_j, u):
    Ns = 0
    Nf = 0
    for rec in records:
        if rec["sender"] == node_i and rec["receiver"] == node_j:
            if rec["success"] == 1:
                Ns += 1
            else:
                Nf += 1
    total = Ns + Nf
    if total == 0:
        return default_trust, 0
    bayesian_trust = (Ns + 1) / (total + 2)
    penalty = (1 - u) ** Nf
    direct_trust = bayesian_trust * penalty
    return direct_trust, total



*   Divide the sliding window into periods, compute the direct trust for each period, and weight them by an exponential decay function.
*   Returns:
 *    overall_direct_trust: the weighted average direct trust.
 *    weighted_interactions: the sum of decay-weighted interaction counts (proxy for confidence).







In [ ]:
def compute_direct_trust_periods(records, node_i, node_j, current_time, T_window, n_periods, decay_rate, u):
    boundaries = get_period_boundaries(current_time, T_window, n_periods)
    weighted_trust_sum = 0.0
    decay_sum = 0.0
    weighted_interactions = 0.0

    for (start, end) in boundaries:
        period_mid = (start + end) / 2
        recs = get_records_in_period(records, start, end)
        period_trust, count = compute_direct_trust_in_period(recs, node_i, node_j, u)
        decay = period_decay(period_mid, current_time, decay_rate)
        weighted_trust_sum += decay * period_trust
        decay_sum += decay
        weighted_interactions += decay * count
    if decay_sum == 0:
        overall_direct_trust = default_trust
    else:
        overall_direct_trust = weighted_trust_sum / decay_sum
    return overall_direct_trust, weighted_interactions

* Compute recommended trust for node_j from node_i via third-party nodes.
* Third-party nodes are those that have received messages from node_i and sent messages to node_j.
* Only candidates with sufficient confidence (weighted interactions >= confidence_threshold) are considered.
* Returns the sum of their direct trust values toward node_j, or -1 if no candidate qualifies.

In [ ]:
def compute_recommended_trust(records, node_i, node_j, current_time, T_window, n_periods, decay_rate, u, confidence_threshold):
    # Candidate nodes that received from node_i and sent to node_j
    candidates_from_i = {rec["receiver"] for rec in records if rec["sender"] == node_i and (current_time - T_window) <= rec["time"] <= current_time}
    candidates_to_j = {rec["sender"] for rec in records if rec["receiver"] == node_j and (current_time - T_window) <= rec["time"] <= current_time}
    candidate_nodes = candidates_from_i.intersection(candidates_to_j)

    if not candidate_nodes:
        return -1

    rec_trust_sum = 0.0
    candidate_count = 0
    for k in candidate_nodes:
        trust_ik, confidence_ik = compute_direct_trust_periods(records, node_i, k, current_time, T_window, n_periods, decay_rate, u)
        if confidence_ik >= confidence_threshold:
            trust_kj, _ = compute_direct_trust_periods(records, k, node_j, current_time, T_window, n_periods, decay_rate, u)
            rec_trust_sum += trust_kj
            candidate_count += 1
    if candidate_count == 0:
        return -1
    return rec_trust_sum



* Compute the integrated trust from node_i to node_j.
* If the confidence (weighted interaction count) is high, use direct trust.
* Otherwise, fuse direct trust with recommended trust using the weight alpha.



In [ ]:
def compute_integrated_trust(records, node_i, node_j, current_time, T_window, n_periods, decay_rate, u, confidence_threshold, alpha, default_trust):
    direct_trust, confidence = compute_direct_trust_periods(records, node_i, node_j, current_time, T_window, n_periods, decay_rate, u)
    if confidence >= confidence_threshold:
        return direct_trust
    else:
        rec_trust = compute_recommended_trust(records, node_i, node_j, current_time, T_window, n_periods, decay_rate, u, confidence_threshold)
        effective_alpha = alpha if rec_trust >= 0 else 0.0
        integrated_trust = (1 - effective_alpha) * direct_trust + effective_alpha * rec_trust
        return integrated_trust

In [ ]:
node_source = input("Enter source node ranging from A to Z: ").upper()
node_candidate = input(f"Enter last node ranging from A to Z excluding {node_source}: ").upper()
if(node_source==node_candidate):
  print("source node is same as last node")
else:
  integrated_trust_value = compute_integrated_trust(records, node_source, node_candidate,
                                                  current_time, T_window, n_periods,
                                                  decay_rate, u, confidence_threshold, alpha, default_trust)

  print(f"Integrated trust from {node_source} to {node_candidate}: {integrated_trust_value:.3f}")

Enter source node ranging from A to Z: A
Enter last node ranging from A to Z excluding A: B
Integrated trust from A to B: 0.674
